In [21]:
import boto3
import pandas as pd
import os

Exercise 1: Automate One File Upload

In [2]:
## task 1 - connect to s3 using boto3

#tell boto3 to use the default profile
session = boto3.Session(profile_name='default',region_name='us-west-2')
#tell boto3 to use the default region
ec2 = session.resource('ec2')


In [ ]:
## task 2 - create a bucket

yourname= input('Please enter the your name: ')
# Create an S3 client
s3_client = session.client('s3')
bucket_name = f'student-{yourname}-bucket'
s3_client.create_bucket(
    Bucket=bucket_name,
    CreateBucketConfiguration={
        'LocationConstraint': session.region_name
    }
)
print(f'Bucket {bucket_name} created in {session.region_name} region')

In [ ]:
##Task 3. upload a file to the bucket

local_file_name= './team_image2.png'
s3_file_name = 'team_image2.png'
s3_client.upload_file(local_file_name, bucket_name, s3_file_name)
# s3_client.upload_file(local_file_name, bucket_name, s3_file_name)
print(f'File {local_file_name} uploaded to bucket {bucket_name} with name {s3_file_name}')

In [ ]:
#task 4. list the files in the bucket
response = s3_client.list_objects_v2( Bucket=bucket_name )
#list all buckets
print('List of buckets:')
if 'Contents' in response:
    for bucket in response['Contents']:
        print(bucket['Key'])
print (f'Total number of files in the bucket: {len(response["Contents"])}')


Exercise 2: Upload Multiple Files Automatically

In [ ]:
#task 1. check if bucket exists
yourname= input('Please enter the your name: ')

# Create an S3 client
s3_client = session.client('s3')

# Check if the bucket exists
bucket_name = f'student-{yourname}-bucket'

def bucket_exists(bucket_name):
    # s3_client = boto3.client('s3')
    response = s3_client.list_buckets()
    buckets = [bucket['Name'] for bucket in response.get('Buckets', [])]
    # print (f'List of buckets: {buckets}')
    return bucket_name in buckets
bucket_exists (bucket_name)

if bucket_exists(bucket_name):
    print(f'Bucket {bucket_name} exists.')
else:
    s3_client.create_bucket(
    Bucket=bucket_name,
    CreateBucketConfiguration={
        'LocationConstraint': session.region_name
    }    
    )
    print(f'Bucket {bucket_name} created in {session.region_name} region')




In [31]:
local_file_name= 'text_local_2.txt'
s3_file_name = 'text.txt'
s3_client.download_file(bucket_name, s3_file_name, local_file_name)

In [ ]:
##task 2. upload files from a directory to the bucket


def upload_files_to_bucket(bucket_name, local_directory):
   
   
    # if the directory dosent have folders:
    # file_2_upload = os.listdir(f'./{local_directory}')
    # if the directory has folders:
    file_list = []
    print ('Starting upload of daily documents')
    for _, _, files in os.walk('./daily_documents'):
        for file in files:
            file_list.append(file)

    for file in file_list:
        local_file_name= f'./{local_directory}/{file}'
        s3_file_name = f'{file}'
        s3_client.upload_file(local_file_name, bucket_name, s3_file_name)
        print (f'File {local_file_name} uploaded to bucket {bucket_name} with name {s3_file_name}')
    print ('Upload of daily documents completed')
    print (f'Total number of files uploaded: {len(file_list)}')
        

# s3_client.upload_file(local_file_name, bucket_name, s3_file_name)
# print(f'File {local_file_name} uploaded to bucket {bucket_name} with name {s3_file_name}')

local_directory= 'daily_documents'
upload_files_to_bucket(bucket_name, local_directory)

Exercise 3 (Advanced): Auto-Skip Already Uploaded Files

In [ ]:

def upload_files_to_bucket(bucket_name, local_directory):
    # files in bucket
    response = s3_client.list_objects_v2( Bucket=bucket_name )
    files_in_bucket = []
    if 'Contents' in response:
        for bucket in response['Contents']:
            files_in_bucket.append(bucket['Key'])
    
    # files in local directory
    file_in_local = []
    for _, _, files in os.walk('./daily_documents'):
        for file in files:
            file_in_local.append(file)
    # check if the files in local directory are already in the bucket
    print ('Starting upload of daily documents')
    counter = 0
    for file in file_in_local:
        if file in files_in_bucket:
            # file_in_local.remove(file)
            print (f'File {file} already exists in the bucket')
            
        else:
            s3_client.upload_file(local_file_name, bucket_name, s3_file_name)
            print (f'File {local_file_name} uploaded to bucket {bucket_name} with name {s3_file_name}')
            counter += 1
            

    print ('Upload of daily documents completed')
    print (f'Total number of files uploaded: {counter}')


local_directory= 'daily_documents'
upload_files_to_bucket(bucket_name, local_directory)
    
